In [32]:
#use Pandas to import data
import pandas as pd
import zipfile
import requests
import io
import time
import numpy as np

In [2]:
## information on filename
citidir = "https://s3.amazonaws.com/tripdata/"
citi_extension = "-citibike-tripdata"
year = 2015
months = ['01','02','03','04','05','06','07','08','09','10','11','12']

In [3]:
## store all data first in dictionary format using months as keys
d = {}

## download monthly data from the web for each month
for mth in months:
    #create filenames for given month
    filename = str(year) + mth + citi_extension
    citibike_zip_filename = citidir + filename + ".zip"
    citibike_csv_filename = filename + ".csv"
    print(citibike_zip_filename)
    
    #find file and use read_csv to save in dictionary of dataframes
    requested_file = requests.get(citibike_zip_filename)
    z = zipfile.ZipFile(io.BytesIO(requested_file.content))
    d[mth] = pd.read_csv(z.open(citibike_csv_filename), header=0, sep=',', quotechar='"')

https://s3.amazonaws.com/tripdata/201501-citibike-tripdata.zip
https://s3.amazonaws.com/tripdata/201502-citibike-tripdata.zip
https://s3.amazonaws.com/tripdata/201503-citibike-tripdata.zip
https://s3.amazonaws.com/tripdata/201504-citibike-tripdata.zip
https://s3.amazonaws.com/tripdata/201505-citibike-tripdata.zip
https://s3.amazonaws.com/tripdata/201506-citibike-tripdata.zip
https://s3.amazonaws.com/tripdata/201507-citibike-tripdata.zip
https://s3.amazonaws.com/tripdata/201508-citibike-tripdata.zip
https://s3.amazonaws.com/tripdata/201509-citibike-tripdata.zip
https://s3.amazonaws.com/tripdata/201510-citibike-tripdata.zip
https://s3.amazonaws.com/tripdata/201511-citibike-tripdata.zip
https://s3.amazonaws.com/tripdata/201512-citibike-tripdata.zip


In [4]:
## Concatenate into single yearly dataframe
df = pd.DataFrame()

## append results of each month
for mth in months:
    df = df.append(d[mth], ignore_index=True)

In [6]:
#check out headers
print(df.tail())

         tripduration            starttime             stoptime  \
9937964           509  12/31/2015 16:50:17  12/31/2015 16:58:46   
9937965           591  12/31/2015 17:17:52  12/31/2015 17:27:43   
9937966           197  12/31/2015 17:56:22  12/31/2015 17:59:39   
9937967           372  12/31/2015 21:20:38  12/31/2015 21:26:50   
9937968           395  12/31/2015 21:49:19  12/31/2015 21:55:55   

         start station id          start station name  start station latitude  \
9937964              3242  Schermerhorn St & Court St               40.691029   
9937965              3242  Schermerhorn St & Court St               40.691029   
9937966              3242  Schermerhorn St & Court St               40.691029   
9937967              3242  Schermerhorn St & Court St               40.691029   
9937968              3242  Schermerhorn St & Court St               40.691029   

         start station longitude  end station id             end station name  \
9937964               -73.991

In [7]:
print("Median trip duration:", df["tripduration"].median())

Median trip duration: 629.0


In [8]:
#add new column to dataframe indicating whether start and end station IDs are the same or not
df['same_start_end'] = pd.Series(df["start station id"] == df["end station id"], index=df.index)
same_frac = df['same_start_end'].value_counts(normalize = True)
print("Fraction of rides that start and end at same station:", same_frac[True])

Fraction of rides that start and end at same station: 0.0223583913373


In [27]:
### count number of unique biks
unique_bikes = df['bikeid'].unique()
print(len(unique_bikes))

8477


In [61]:
bikes = df.groupby('bikeid')

In [78]:
visited_stations = {}

for k,v in bikes:
    start_stations = bikes.get_group(k)['start station id']
    end_stations = bikes.get_group(k)['end station id']
    stations = pd.concat([start_stations, end_stations], axis=0, ignore_index = True)
    unique_stations = stations.unique()
    visited_stations[k] = len(unique_stations)

In [81]:
np.std(list(visited_stations.values()))

54.541896535851848

In [93]:
len(vis)

8477

In [102]:
vis = list(visited_stations.values())
print(np.mean(vis))
print(np.median(vis))
print(np.std(vis))
np.sort(vis)[1000]

320.790020054
333.0
54.5418965359


array([  1,   1,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,
         2,   2,   3,   3,   3,   3,   3,   3,   4,   4,   4,   4,   4,
         4,   4,   4,   4,   5,   5,   5,   6,   6,   6,   6,   6,   7,
         7,   7,   8,   8,   8,   8,   9,   9,   9,  10,  11,  11,  11,
        11,  11,  12,  12,  13,  13,  13,  14,  14,  14,  16,  17,  17,
        17,  19,  21,  22,  22,  23,  26,  26,  26,  26,  27,  28,  29,
        30,  34,  34,  35,  35,  35,  37,  39,  39,  41,  42,  45,  45,
        46,  46,  47,  48,  49,  51,  51,  52,  53,  53,  54,  54,  54,
        55,  56,  56,  58,  58,  59,  60,  61,  62,  62,  63,  63,  63,
        63,  63,  64,  65,  65,  65,  66,  66,  67,  68,  68,  69,  70,
        70,  74,  74,  75,  76,  76,  78,  79,  80,  80,  81,  81,  83,
        83,  84,  84,  85,  85,  89,  90,  90,  91,  91,  93,  93,  93,
        93,  95,  95,  96,  96,  97,  97,  98,  98, 100, 100, 100, 100,
       102, 103, 104, 104, 104, 104, 104, 105, 106, 106, 107, 10

In [70]:
type(end_stations)

pandas.core.series.Series

In [60]:
bikes['test_column'] = bikes['start station id']*2

TypeError: unsupported operand type(s) for *: 'SeriesGroupBy' and 'int'

In [13]:
start_stations = bikes['start station id']
end_stations = bikes['end station id']

In [57]:
for k,v in start_stations:
    print(type(k))
    print(type(v))
    time.sleep(3)

<class 'numpy.int64'>
<class 'pandas.core.series.Series'>
<class 'numpy.int64'>
<class 'pandas.core.series.Series'>
<class 'numpy.int64'>
<class 'pandas.core.series.Series'>


KeyboardInterrupt: 

In [22]:
for k,v in start_stations:
    print(k)
    print(len(v))
    time.sleep(3)

14529
1233
14530
588
14531
1081
14532
1088
14533
1326
14534
1308


KeyboardInterrupt: 

In [43]:
list(start_stations)[0][1]

1524        326
1793        497
5715        496
8123        195
8360        250
8642        358
8812        435
9156        466
29006       362
40268       456
41290       498
42450       474
46917       478
49233       520
52693       379
64762       468
66571       465
69884       348
70780       128
71709       508
74789       362
74959       167
92747       465
93800       359
94762       479
95286       512
95408       489
95589       515
95984       434
97689       357
           ... 
9673414     499
9673468     499
9683359     503
9691099     505
9702844     510
9704054     511
9704064     511
9710968     513
9718734     517
9721447     517
9727077     519
9732240     519
9738998     521
9740958     521
9742686     522
9748180     523
9767093     531
9771969     533
9775500     536
9778113     537
9785418     540
9786726     545
9856904    3134
9860888    3137
9860914    3137
9862644    3139
9891913    3160
9904603    3168
9912224    3175
9926818    3230
Name: start station id, 

In [47]:
bikes.get_group(14529)['start station id'].unique()

array([ 326,  497,  496,  195,  250,  358,  435,  466,  362,  456,  498,
        474,  478,  520,  379,  468,  465,  348,  128,  508,  167,  359,
        479,  512,  489,  515,  434,  357,  469,  490,  352, 2022,  447,
        495,  237,  402,  280,  342,  531,  296,  249,  540,  476,  393,
        300,  336, 2010,  368,  463,  127,  382,  223,  293,  403,  317,
        151,  238,  477,  526,  488,  380,  455,  267,  513,  404,  327,
        521,  533,  507,  236,  325,  459,  426,  347,  294,  523,  546,
        445,  517,  537,  464,  433,  312,  518,  164,  538,  458,  530,
        328,  318, 2006,  499,  461,  334,  264,  504,  411,  301,  491,
        511, 2012,  501,  522,  510,  441,  485,  305,  480,  484,  446,
       2021,  310,  406,  376,  146,  408,  505,  375,  536,  525,  448,
        259,  400,  303,  152,  306,  276,  316,  415,  279, 2004,  252,
        503,  516,  308,  363,  346,  383,  492,  487,  168,  527,  509,
        483,  297,  438,  401,  356,  229,   79,  2

In [39]:
start_groups = start_stations[:][1].groups

Exception: Column(s) start station id already selected

In [28]:
for k,v in start_groups.items():
    print(k)
    print(v)
    time.sleep(3)

16384
Int64Index([ 310666,  310792,  310827,  311876,  311957,  312174,  312284,
             317817,  327540,  329184,
            ...
            9871878, 9876316, 9887862, 9891524, 9894988, 9900518, 9901120,
            9918194, 9926187, 9929251],
           dtype='int64', length=1261)
16385
Int64Index([  57492,   57562,   59694,   60688,   63911,   64175,   67342,
              70718,  150859,  154102,
            ...
            9864457, 9864462, 9867593, 9873595, 9878460, 9881575, 9907128,
            9909383, 9918169, 9933977],
           dtype='int64', length=980)


KeyboardInterrupt: 

In [25]:
len(start_groups.keys())

8477